In [36]:
import fitz
import random
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

device = 'mps'

In [75]:
import nest_asyncio
nest_asyncio.apply()
import os
os.environ['LLAMA_CLOUD_API_KEY'] = ''
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data('lbg_relationship_tnc.pdf')

llama_text = documents[0].text

Started parsing the file under job_id ab524aca-2acd-4a6d-ba1e-365d39d168c4


In [79]:
llama_text_n_pages = []
for page_n, page in enumerate(llama_split):
    llama_text_n_pages.append({
        'page_n': page_n,
        'sentence_chunk': page
    })

In [37]:
pdf_path = 'lbg_relationship_tnc.pdf'
# lbg_relationship_tnc.pdf account_bank_tnc.pdf
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_n_texts = []

    for page_n, page in enumerate(doc):
        text = page.get_text()
        # text = text.replace('\n', ' ').replace('  ', ' ')

        pages_n_texts.append({
            'page_n': page_n,
            'page_char_count': len(text),
            'page_word_count_raw': len(text.split(' ')),
            'page_sentence_count_raw': len(text.split('. ')),
            'page_token_count': len(text) / 4, # 1 token ~= 4 chars
            'text': text
        })

    return pages_n_texts

pages_n_texts = open_and_read_pdf(pdf_path)

In [38]:
random.sample(pages_n_texts, k=1)

[{'page_n': 24,
  'page_char_count': 2754,
  'page_word_count_raw': 450,
  'page_sentence_count_raw': 5,
  'page_token_count': 688.5,
  'text': '21\nCore Banking Agreement\n14. Termination\n14.1 You may stop receiving a Product from us or end your \nrelationship with us entirely at any time for any reason.  \nIf you wish to do so, you will need to give one month’s \nnotice to us. Such notice needs to be given to us in writing \nby post, unless otherwise agreed between you and us. \n14.2 We may stop providing you with a Product or end our \nrelationship with you entirely:\n14.2.1 at any time for any reason by giving two months’ \nwritten notice to you (provided that we also give you \nany other notice that we are required to give to you \nby law); and \n14.2.2 immediately, if any of the circumstances set out \nbelow has occurred (and we will provide you with \nsuch notice (if any) that we are required by law to \nprovide to you):\n14.2.2.1 you breach The Agreement or any other \nagreeme

In [39]:
df = pd.DataFrame(pages_n_texts)
df.head()

,page_n,page_char_count,page_word_count_raw,page_sentence_count_raw,page_token_count,text
0,0,57,7,1,14.25,RELATIONSHIP \nTERMS & CONDITIONS\nCORE BANKIN...
1,1,0,1,1,0.00,
2,2,1703,270,39,425.75,Contents\n \nImportant information \n \n1\n1 G...
3,3,254,39,3,63.50,Core Banking Agreement\n(“The Agreement”) cont...
4,4,1063,153,2,265.75,Core Banking Agreement\n1\nImportant Informati...


In [40]:
df.describe().round(2)

,page_n,page_char_count,page_word_count_raw,page_sentence_count_raw,page_token_count
count,56.00,56.00,56.00,56.00,56.00
mean,27.50,2033.82,332.39,7.29,508.46
std,16.31,1482.99,246.65,6.55,370.75
min,0.00,0.00,1.00,1.00,0.00
25%,13.75,274.25,41.25,2.00,68.56
50%,27.50,2254.00,381.50,6.00,563.50
75%,41.25,3136.25,517.50,11.00,784.06
max,55.00,4461.00,757.00,39.00,1115.25


In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)

pages_n_chunks_new = []

for item in pages_n_texts:
    item['sentence_chunks'] = text_splitter.split_text(item['text'])
    for chunk in item['sentence_chunks']:
        chunk_dict = {}
        chunk_dict['page_n'] = item['page_n']
        # print('\nNEW CHUNK: ', chunk)
        # joined_sentence_chunk = ''.join(chunk).replace('  ', ' ').strip()
        # joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        # joined_sentence_chunk = re.sub(r'\d+(\.\d+)+', '', joined_sentence_chunk)
        # print('\nMODIFIED CHUNK: ', joined_sentence_chunk)
        chunk_dict['sentence_chunk'] = chunk#joined_sentence_chunk

        # # add metadata
        # chunk_dict['chunk_chars'] = len(joined_sentence_chunk)
        # chunk_dict['chunk_words'] = len([word for word in joined_sentence_chunk.split(' ')])
        # chunk_dict['chunk_tokens'] = len(joined_sentence_chunk) / 4

        pages_n_chunks_new.append(chunk_dict)

len(pages_n_chunks_new)

176

In [42]:
llama_split = text_splitter.split_text(llama_text)

In [48]:
# len(llama_split)
# type(pages_n_chunks_new[0]), type(llama_split[0])
pages_n_chunks_new[0]

{'page_n': 0,
 'sentence_chunk': 'RELATIONSHIP \nTERMS & CONDITIONS\nCORE BANKING  AGREEMENT'}

### Embedding chunks

In [7]:
pages_n_chunks_new

[{'page_n': 0,
  'sentence_chunk': 'RELATIONSHIP \nTERMS & CONDITIONS\nCORE BANKING  AGREEMENT'},
 {'page_n': 2,
  'sentence_chunk': 'Contents\n \nImportant information \n \n1\n1 General \n1.  Information about our relationship with you \n5\n2.  Definitions and interpretation \n6\n3.  Your relationship with us \n9\n4.  Providing services to you \n9\n5.  Your warranties \n9\n6.  Who is authorised to give instructions to us \n11\n7.  Confidentiality \n12\n8.  Changes to the Terms and Conditions \n13\n9.  Terms applying to charges \n14\n10.  Interest paid and charged \n16\n11. Third Party Providers\n12.  Partnerships \n18\n13.  Suspension of a product \n19\n14.  Termination \n21\n15.  Your rights to cancel \n24\n16.  What happens after termination or cancellation \n24\n17.  Liability \n25\n18.  Circumstances beyond your or our control \n26\n19.  Set off \n26\n20.  Other terms you need to know about your agreement with us \n28'},
 {'page_n': 2,
  'sentence_chunk': '21.  Information about u

In [49]:
emb_model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1').to(device)
#mixedbread-ai/mxbai-embed-large-v1 all-mpnet-base-v2
text_chunks = [item['sentence_chunk'] for item in pages_n_chunks_new]
len(text_chunks)

Python(11812) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


176

In [55]:
# text_chunk_embs = emb_model.encode(text_chunks, batch_size=16, convert_to_tensor=True)
llama_chunk_embs = emb_model.encode(llama_split, batch_size=16, convert_to_tensor=True)
# text_chunk_embs.shape, text_chunk_embs
llama_chunk_embs.shape, llama_chunk_embs

(torch.Size([198, 1024]),
 tensor([[-0.2894, -0.6762, -0.2548,  ..., -0.2197,  0.0575,  0.2950],
         [-0.3220, -0.8583, -0.5748,  ..., -0.4800, -0.3515,  0.4240],
         [-0.0929, -0.4366, -0.6115,  ..., -0.1398, -0.7231,  0.4906],
         ...,
         [-0.8480, -0.5612, -0.6425,  ..., -0.5232, -0.2608,  0.8832],
         [-0.7390, -1.1355, -0.4723,  ..., -0.5159, -0.4490,  0.7281],
         [-0.2831,  0.1584, -0.2007,  ..., -0.0353,  0.0897,  0.0589]],
        device='mps:0'))

In [80]:
llama_text_n_pages[0]

{'page_n': 0,
 'text': 'CORE BANKING AGREEMENT RELATIONSHIP TERMS & CONDITIONS\n---\nNO_CONTENT_HERE\n---\n|Contents|Page|\n|---|---|\n|Important information|1|\n|General| |\n|Information about our relationship with you|5|\n|Definitions and interpretation|6|\n|Your relationship with us|9|\n|Providing services to you|9|\n|Your warranties|9|\n|Who is authorised to give instructions to us|11|\n|Confidentiality|12|\n|Changes to the Terms and Conditions|13|\n|Terms applying to charges|14|\n|Interest paid and charged|16|\n|Third Party Providers| |\n|Partnerships|18|\n|Suspension of a product|19|\n|Termination|21|\n|Your rights to cancel|24|\n|What happens after termination or cancellation|24|\n|Liability|25|\n|Circumstances beyond your or our control|26|\n|Set off|26|\n|Other terms you need to know about your agreement with us|28|'}

In [81]:
pages_n_chunks_new[0]

{'page_n': 0,
 'sentence_chunk': 'RELATIONSHIP \nTERMS & CONDITIONS\nCORE BANKING  AGREEMENT'}

In [82]:
emb_chunks_df = pd.DataFrame(llama_text_n_pages)
embs_only_df = pd.DataFrame(llama_chunk_embs.to('cpu'))
emb_chunks_df['embedding'] = embs_only_df.values.tolist()
emb_df_save_path = 'emb_chunks_df.csv'
emb_chunks_df.to_csv(emb_df_save_path, index=False)

In [83]:
emb_chunks_df['embedding']

0      [-0.289400577545166, -0.6761614680290222, -0.2...
1      [-0.322034627199173, -0.8583006262779236, -0.5...
2      [-0.09293383359909058, -0.43664729595184326, -...
3      [-0.36704424023628235, -0.7181680202484131, 0....
4      [-0.19214218854904175, -0.26558810472488403, -...
                             ...                        
193    [0.06225999817252159, -0.01736411452293396, 0....
194    [-0.6477031707763672, -0.5587860941886902, 0.0...
195    [-0.8480250239372253, -0.5612276196479797, -0....
196    [-0.7389596700668335, -1.1355124711990356, -0....
197    [-0.283149778842926, 0.15843471884727478, -0.2...
Name: embedding, Length: 198, dtype: object

In [12]:
emb_chunks_df.describe().round(2)

,page_n
count,176.00
mean,29.01
std,14.63
min,0.00
25%,17.00
50%,29.00
75%,41.00
max,55.00
